#### Make metadata

Information from PDF reports from BaseClear combined copied into csv files (data/metadata/Run1 .../Run3). 
Metadata cleaned and mearged with shedding metadata. 

In [115]:
source("scr/functions/general/load_abs_install_pkg.R")
load_abs_install_pkg(c("tidyverse", "lubridate"))

[[1]]
[1] TRUE

[[2]]
[1] TRUE

In [116]:
#Clean and prepera metadata for Run 1
#######################################
m1 <- read.csv("data/metadata/Run1.csv")
m1 <- mutate_all(m1, as.character)
m1[,1] <- sub(" X", "", m1[,1])
m1.t <- m1[m1[,3] %in% "Target", ]
m1.t[,1] <- gsub("-", "_", m1.t[,1]) 
m1.t <- cbind(m1.t, str_split(string = m1.t[,1], pattern = "_", simplify = TRUE))
colnames(m1.t) <- c("SampleID", "Run", "SampleType", "CowN", "Day", "Month", "Year")

m1.t <- mutate_all(m1.t, as.character)

m1.t$Month <- ifelse(nchar(m1.t$Month)==1, paste0("0", m1.t$Month), m1.t$Month)

m1.t$Day <- ifelse(nchar(m1.t$Day) == 1, paste0("0", m1.t$Day), m1.t$Day)

m1.t$Year <- ifelse(m1.t$Year == "99", paste0("19", m1.t$Year), m1.t$Year)
m1.t$Year <- ifelse(nchar(m1.t$Year) == 2, paste0("20", m1.t$Year), m1.t$Year)
m1.t$Year <- ifelse(nchar(m1.t$Year) == 1, paste0("200", m1.t$Year), m1.t$Year)

m1.t$CleanID <- paste(m1.t$Year, m1.t$Month, m1.t$Day, m1.t$CowN, sep = "_")

m1.full <- full_join(m1.t, m1[!m1[,3] %in% "Target", ])
m1.full[is.na(m1.full)] <- "Controls"


Joining, by = c("SampleID", "Run", "SampleType")


In [117]:
#Clean and prepera metadata for Run 2
#######################################

m2 <- read.csv("data/metadata/Run2.csv")
m2 <- mutate_all(m2, as.character)
m2[,1] <- sub(" X", "", m2[,1])

m2.t <- m2[m2[,3] %in% "Target", ]
m2.t <- cbind(m2.t, str_split(string = m2.t[,1], pattern = "_", simplify = TRUE))
colnames(m2.t) <- c("SampleID", "Run", "SampleType", "Year", "Month", "Day", "CowN")
m2.t <- mutate_all(m2.t, as.character)
m2.t$CleanID <- m2.t$SampleID

m2.full <- full_join(m2.t, m2[!m2[,3] %in% "Target", ])
m2.full[is.na(m2.full)] <- "Controls"


Joining, by = c("SampleID", "Run", "SampleType")


In [127]:
#Clean and prepera metadata for Run 3
#######################################

m3 <- read.csv("data/metadata/Run3.csv")

m3[,1] <- sub(" X", "", m3[,1])

m3.t <- mutate_all(m3, as.character)

m3.t$WellID <- as.character(sub(".*-", "", m3.t$SampleID))


### Read in and clean the metadata ###
m3.new <- read.csv("data/metadata/meta_lars_new.csv", header = FALSE, stringsAsFactors = FALSE)

m3.new <- m3.new[2:nrow(m3.new), ]

m3.new$V4 <- gsub("/", "-", m3.new$V4)

# Split data column and format day and month in to two digit format
m3.new.date <- str_split(string = m3.new$V4, pattern = "-", simplify = TRUE)

m3.new.date[,1] <- ifelse(nchar(m3.new.date[,1]) == 1, paste0("0", m3.new.date[,1]), m3.new.date[,1]) 

m3.new.date[,2] <- ifelse(nchar(m3.new.date[,2]) == 1, paste0("0", m3.new.date[,2]), m3.new.date[,1]) 

# Combine dataframes 
m3.new.full <- cbind(m3.new, m3.new.date)

colnames(m3.new.full) <- paste0("Col_", colnames(m3.new.full))

m3.new.full$CleanIDNew <- paste(m3.new.full$Col_3, 
                                 m3.new.full$Col_2, 
                                 m3.new.full$Col_1, 
                                 m3.new.full$Col_V3, 
                                 sep="_")

m3.new.full <- m3.new.full[m3.new.full$Col_V1 %in% m3.t$WellID, ]

# Combine illumina and Lars metadata
####################################

# Oreder metadata by Wells ID 
m3.t <- m3.t[order(m3.t$WellID), ]

m3.new.full <- m3.new.full[order(m3.new.full$Col_V1), ]


m3.full <- cbind(m3.t, m3.new.full)[,c(1, 2, 3, 7, 10, 11, 12, 13)]

colnames(m3.full) <- c("SampleID", "Run", "SampleType", "CowN", "Day", "Month", "Year", "CleanID")


In [119]:
# Combine metadata from all 3 runs 

m.comb <- rbind(m1.full, m2.full, m3.full)

m.comb.samp <- m.comb[!m.comb$CleanID %in% "Controls", ]

m.comb.samp$CowN <- paste0("C", m.comb.samp$CowN)

In [120]:
# Read in shedding data 
# add columns for merging with rest of data
######################

m.shedding <- read.csv("output/1_shedding_analysis/shedding_data_long.csv")

m.shedding <- cbind(m.shedding, str_split(m.shedding$SDATE, pattern = "_", simplify = TRUE))[, -1]

colnames(m.shedding) <- c("SDATE", "CowN_shed", "SheddingScore", "Year_shed", "Month_shed", "Day_shed")

In [121]:
# Due to discripancy in the date between 
# samples marking and shedding data we combine 
# metadata based on the closest day 
############################################

# Set date of day zero
startdate <- as.Date("08/02/1999","%d/%m/%Y")


# Calculate number of days and add it as a column for shedding data
###################################################################
shedd.date <- ymd(paste(m.shedding$Year_shed, 
                    m.shedding$Month_shed, 
                    m.shedding$Day_shed, sep= "/"))

m.shedding$AgeDays_shed <- as.numeric(as.character(difftime(time1 = shedd.date, 
                                                            startdate, 
                                                            units = "days")))

# Age in month 
m.shedding$AgeMonth <- round(as.numeric(as.character(difftime(time1 = shedd.date, 
                                                              startdate, units = "weeks"))) / 4) + 1

# Add diet type based on age
m.shedding$Diet <- ifelse(m.shedding$AgeMonth == 1, "Milk", "Solid")

# Calculate number of days and add it as a column for metadata
##############################################################
samp.day <- paste(m.comb.samp$Year, m.comb.samp$Month, m.comb.samp$Day)

samp.day <- ymd(samp.day)

m.comb.samp$AgeDays <- as.numeric(as.character(difftime(time1 = samp.day, 
                                                        startdate, units = "days")))

In [122]:
# Find closest to metadata dates in shedding dates 
# Will be found based on age in days per cow 

m.shedding <- mutate_all(m.shedding, as.character)
m.comb.samp <- mutate_all(m.comb.samp, as.character)
empty.sub <- m.shedding[1,]
empty.sub[1,] <- "No_match"

# Every sample have to be checked 
nearest.match.d <- c()
s.all <- c()
for (s in 1:nrow(m.comb.samp)){
    
    # Subset shedding data leaving only samples from  animal in current alteration
    shedd.sub <- m.shedding[m.shedding$CowN_shed %in% m.comb.samp$CowN[s], ]
    
    shedd.day <- as.numeric(as.character(shedd.sub$AgeDays_shed))
    meta.day <- as.numeric(as.character(m.comb.samp$AgeDays[s]))
    
    # Find best matching day 
    sub.d.nearest <- shedd.sub[which.min(abs(shedd.day - meta.day)), ]
    
    if (nrow(sub.d.nearest) !=1) {
        sub.d.nearest <- empty.sub
    }
    togather.d <- cbind(m.comb.samp[s, ], sub.d.nearest)
    
    nearest.match.d <- rbind(nearest.match.d, togather.d)
    
    s.all <- rbind(s.all, m.comb.samp[s, ])
}

In [123]:
# Combine target and control samples 
# and write final metadata csv. 

meta.final <- full_join(nearest.match.d, m.comb[m.comb$CleanID %in% "Controls", ])

meta.final$SampleID <- sub("neg_A12", "neg_57195-A12", meta.final$SampleID)
meta.final$SampleID <- sub("neg_H11", "neg_57195-H11", meta.final$SampleID)

meta.final[is.na(meta.final)] <- "Controls"

meta.final <- meta.final[,c('SampleID', 'CowN_shed', "SDATE",  
                            'Year_shed', 'Month_shed', 'Day_shed', 
                            'SheddingScore', "AgeMonth", "Diet")]

colnames(meta.final) <- c('OriginalID', 'CowN', "SDATE",  'Year', 
                          'Month', 'Day', 'SheddingScore', "AgeMonth", "Diet")


Joining, by = c("SampleID", "Run", "SampleType", "CowN", "Day", "Month", "Year", "CleanID")


In [124]:
# Find and remove duplicates (manualy)
meta.final$NewId <- paste(meta.final$CowN, meta.final$SDATE, sep="_")

meta.dup <- meta.final[meta.final$NewId %in% meta.final$NewId[duplicated(meta.final$NewId)], ]

# Inspected manually 
# meta.dup[order(meta.dup$NewId), ]

# Add a column - files to keep 
s.rem <- c("1359_14-5-01", "1360_19-5-03")

meta.final$FilesKeep <- ifelse(meta.final$CowN == "Controls", "Remove", "Keep")

meta.final$FilesKeep[meta.final$OriginalID %in% s.rem] <- "Remove"

In [125]:
dir.create("data/Dada2_metadata")
write.csv(meta.final, "data/metadata/samples_metadata.csv")

Warning message in dir.create("data/Dada2_metadata"):
“'data/Dada2_metadata' already exists”